# Download all the Python Libraries

In [1]:
# Check if the copmuter is on google colab
import sys
if 'google.colab' in sys.modules:
    print("Running on Google Colab")
    !pip install rich
    !pip install -q -U bitsandbytes
    !pip install -q -U git+https://github.com/huggingface/transformers.git 
    !pip install -q -U git+https://github.com/huggingface/peft.git
    !pip install -q -U git+https://github.com/huggingface/accelerate.git
    !pip install datasets
    !pip install wandb
else:
    print("Not running on Google Colab")
from rich import print
import logging
from pathlib import Path
logger = logging.getLogger(__name__)
#ROOT_PATH = Path(__file__).parent.parent

Not running on Google Colab


# Check the GPU env
1. You can check the GPU in the Google Colab by clicking  and efficieny
2. Check if the GPU can use bfloat16 most effective as most model are pre-trained with bfloat16

In [2]:
import torch
from rich import print
if torch.cuda.is_available():
    !nvidia-smi
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print("Cuda capability: ", torch.cuda.get_device_capability(0))
    '''
    On pre-ampere hardware bf16 works, but doesn't provide speed-ups compared to fp32 matmul operations, and some matmul operations are failing outright, so this check is more like "guaranteed to work and be performant" than "works somehow".  https://github.com/pytorch/pytorch/issues/75427
    '''
    print(f"bfloat16 support: { torch.cuda.is_bf16_supported()}") 

# Set the Seed Environment of the Notebook to ensure the reproducibility

In [3]:
from transformers import set_seed

DEFAULT_SEED = 42

set_seed( DEFAULT_SEED )

# Constant Variable

In [4]:
MAX_LENGTH  = 1024
BITS  = 4
LR = 1e-5
EPOCHS = 2
# Whether to use bf16 (preferred on A100's)."

# Setup Weight And Bias 💡 Configuration tips

W&B integration with Hugging Face can be configured to add extra functionalities:

* auto-logging of models as artifacts: just set environment varilable `WANDB_LOG_MODEL` to `true`
* log histograms of gradients and parameters: by default gradients are logged, you can also log parameters by setting environment variable `WANDB_WATCH` to `all`
* set custom run names with `run_name` arg present in scripts or as part of `TrainingArguments`
* organize runs by project with the `WANDB_PROJECT` environment variable

For more details refer to [W&B + HF integration documentation](https://docs.wandb.ai/integrations/huggingface).

In [5]:
import wandb
import os

wandb.login()

%env WANDB_LOG_MODEL=true

# set the wandb project where this run will be logged
os.environ["WANDB_PROJECT"]="my-awesome-project"

# save your trained model checkpoint to wandb
#os.environ["WANDB_LOG_MODEL"]="true"

# turn off watch to log faster
os.environ["WANDB_WATCH"]="false"

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: haruqhakim (compress_rl). Use `wandb login --relogin` to force relogin


env: WANDB_LOG_MODEL=true


# Set Up Local Training Root

# Download the Datset from the Hugging Face Datset Face Dataset

In [6]:
DEFAULT_TRAINING_DATASET = "Rami/prompts"
from datasets import load_dataset
training_dataset = load_dataset(
    "Rami/prompts",
)
print(
    training_dataset
)


Found cached dataset parquet (/home/null/.cache/huggingface/datasets/Rami___parquet/Rami--prompts-c0e384db99c3a811/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['context', 'response', 'views', 'instruction', 'category'],
        num_rows: 341
    })
})

# Download the Tokenizers
1. We are suing Dolly model which was trained on the Pythia model. Instead we are recreating the dollvy tokenizer from the Pythia tokenizer

In [7]:
from transformers import AutoTokenizer

# Special Tokens
INSTRUCTION_KEY = "### Instruction:"
INPUT_KEY = "Input:"
RESPONSE_KEY = "### Response:"
END_KEY = "### End"
RESPONSE_KEY_NL = f"{RESPONSE_KEY}\n"
DEFAULT_SEED = 42

PRETRAINED_MODEL_NAME_OR_PATH = "databricks/dolly-v2-3b"
eleutherai_python_3b = "EleutherAI/pythia-2.8b"
dolly_v2_tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL_NAME_OR_PATH)
print(dolly_v2_tokenizer)
pythia_tokenizer = AutoTokenizer.from_pretrained(eleutherai_python_3b)
print(pythia_tokenizer)

# Make sure that the pad token is the end of the tokens
pythia_tokenizer.pad_token = pythia_tokenizer.eos_token

# Add special tokens for End , Instruction , Response Key

pythia_tokenizer.add_special_tokens({
    "additional_special_tokens": [
        END_KEY,
        INSTRUCTION_KEY,
        RESPONSE_KEY,
    ]
})

print(pythia_tokenizer)

GPTNeoXTokenizerFast(name_or_path='databricks/dolly-v2-3b', vocab_size=50254, 
model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', 
special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 
'pad_token': '<|endoftext|>', 'additional_special_tokens': ['### End', '### Instruction:', '### Response:']}, 
clean_up_tokenization_spaces=True)

GPTNeoXTokenizerFast(name_or_path='EleutherAI/pythia-2.8b', vocab_size=50254, 
model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', 
special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}, 
clean_up_tokenization_spaces=True)

GPTNeoXTokenizerFast(name_or_path='EleutherAI/pythia-2.8b', vocab_size=50254, 
model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', 
special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 
'pad_token': '<|endoftext|>', 'additional_special_tokens': ['### End', '### Instruction:', '### Response:']}, 
clean_up_tokenization_spaces=True)

# Process the dataset
1. Convert the dataset into instruction

## Instruct Format String and Constants

In [8]:
INTRO_BLURB = (
    "Below is an instruction that describes a task. Write a response that appropriately completes the request."
)
INSTRUCTION_KEY = "### Instruction:"
INPUT_KEY = "Input:"
RESPONSE_KEY = "### Response:"
END_KEY = "### End"
RESPONSE_KEY_NL = f"{RESPONSE_KEY}\n"
DEFAULT_SEED = 42
# This is a training prompt that does not contain an input string.  The instruction by itself has enough information
# to respond.  For example, the instruction might ask for the year a historic figure was born.
PROMPT_NO_INPUT_FORMAT = """{intro}

{instruction_key}
{instruction}

{response_key}
{response}

{end_key}""".format(
    intro=INTRO_BLURB,
    instruction_key=INSTRUCTION_KEY,
    instruction="{instruction}",
    response_key=RESPONSE_KEY,
    response="{response}",
    end_key=END_KEY,
)

# This is a training prompt that contains an input string that serves as context for the instruction.  For example,
# the input might be a passage from Wikipedia and the intruction is to extract some information from it.
PROMPT_WITH_INPUT_FORMAT = """{intro}

{instruction_key}
{instruction}

{input_key}
{input}

{response_key}
{response}

{end_key}""".format(
    intro=INTRO_BLURB,
    instruction_key=INSTRUCTION_KEY,
    instruction="{instruction}",
    input_key=INPUT_KEY,
    input="{input}",
    response_key=RESPONSE_KEY,
    response="{response}",
    end_key=END_KEY,
)

# This is the prompt that is used for generating responses using an already trained model.  It ends with the response
# key, where the job of the model is to provide the completion that follows it (i.e. the response itself).
PROMPT_FOR_GENERATION_FORMAT = """{intro}

{instruction_key}
{instruction}

{response_key}
""".format(
    intro=INTRO_BLURB,
    instruction_key=INSTRUCTION_KEY,
    instruction="{instruction}",
    response_key=RESPONSE_KEY,
)

In [9]:
from datasets import Dataset, load_dataset

def load_training_dataset(path_or_dataset: str = DEFAULT_TRAINING_DATASET) -> Dataset:
    logger.info(f"Loading dataset from {path_or_dataset}")
    dataset = load_dataset(path_or_dataset)["train"]
    logger.info("Found %d rows", dataset.num_rows)

    def _add_text(rec):
        instruction = rec["instruction"]
        response = rec["response"]
        context = rec.get("context")

        if not instruction:
            raise ValueError(f"Expected an instruction in: {rec}")

        if not response:
            raise ValueError(f"Expected a response in: {rec}")

        # For some instructions there is an input that goes along with the instruction, providing context for the
        # instruction.  For example, the input might be a passage from Wikipedia and the instruction says to extract
        # some piece of information from it.  The response is that information to extract.  In other cases there is
        # no input.  For example, the instruction might be open QA such as asking what year some historic figure was
        # born.
        if context:
            rec["text"] = PROMPT_WITH_INPUT_FORMAT.format(instruction=instruction, response=response, input=context)
        else:
            rec["text"] = PROMPT_NO_INPUT_FORMAT.format(instruction=instruction, response=response)
        return rec

    dataset = dataset.map(_add_text)

    return dataset

## Preprocess the dataset using the tokenizers

In [10]:
import logging
from functools import partial
from pathlib import Path
from typing import Any, Dict, List, Tuple, Union
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    DataCollatorForLanguageModeling,
    PreTrainedTokenizer,
    Trainer,
    TrainingArguments,
    set_seed,
)
def preprocess_batch(batch: Dict[str, List], tokenizer: AutoTokenizer, max_length: int) -> dict:
    return tokenizer(
        batch["text"],
        max_length=max_length,
        truncation=True,
    )
def preprocess_dataset(tokenizer: AutoTokenizer, max_length: int, seed=DEFAULT_SEED, training_dataset: str = DEFAULT_TRAINING_DATASET) -> Dataset:
    """Loads the training dataset and tokenizes it so it is ready for training.

    Args:
        tokenizer (AutoTokenizer): Tokenizer tied to the model.
        max_length (int): Maximum number of tokens to emit from tokenizer.

    Returns:
        Dataset: HuggingFace dataset
    """

    dataset = load_training_dataset(training_dataset)

    logger.info("Preprocessing dataset")
    _preprocessing_function = partial(preprocess_batch, max_length=max_length, tokenizer=tokenizer)
    dataset = dataset.map(
        _preprocessing_function,
        batched=True,
        remove_columns=["instruction", "context", "response", "text", "category"],
    )

    # Make sure we don't have any truncated records, as this would mean the end keyword is missing.
    logger.info("Processed dataset has %d rows", dataset.num_rows)
    dataset = dataset.filter(lambda rec: len(rec["input_ids"]) < max_length)
    logger.info("Processed dataset has %d rows after filtering for truncated records", dataset.num_rows)

    logger.info("Shuffling dataset")
    dataset = dataset.shuffle(seed=seed)

    logger.info("Done preprocessing")

    return dataset

In [22]:
processed_dataset = preprocess_dataset(
    tokenizer = pythia_tokenizer,
    max_length = MAX_LENGTH,
    seed = DEFAULT_SEED,
)

# Split the Dataset

split_dataset = processed_dataset.train_test_split(test_size = 0.2, seed=DEFAULT_SEED, shuffle = True)

train_datset = split_dataset["train"]
print(train_datset)
val_dataset = split_dataset["test"]

logger.info("Train data size: %d", split_dataset["train"].num_rows)
logger.info("Test data size: %d", split_dataset["test"].num_rows)

Found cached dataset parquet (/home/null/.cache/huggingface/datasets/Rami___parquet/Rami--prompts-c0e384db99c3a811/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

Loading cached processed dataset at /home/null/.cache/huggingface/datasets/Rami___parquet/Rami--prompts-c0e384db99c3a811/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-c6fadbd10cdd8d3c.arrow
Loading cached processed dataset at /home/null/.cache/huggingface/datasets/Rami___parquet/Rami--prompts-c0e384db99c3a811/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-c2857169bbcea816.arrow
Loading cached processed dataset at /home/null/.cache/huggingface/datasets/Rami___parquet/Rami--prompts-c0e384db99c3a811/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-35d7446ee0aeeb66.arrow
Loading cached shuffled indices for dataset at /home/null/.cache/huggingface/datasets/Rami___parquet/Rami--prompts-c0e384db99c3a811/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-7891f6c9063637b0.arrow
Loading cached split indices for dataset at /home/null/.cache/huggingface/datasets/Rami___parquet/Rami--p

Dataset({
    features: ['views', 'input_ids', 'attention_mask'],
    num_rows: 272
})

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_504919/1161483090.py:13 in <module>                                               │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_504919/1161483090.py'                       │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'train_dataset' is not defined

## Data Collator For Completion Only LM

In [12]:
import numpy as np
class DataCollatorForCompletionOnlyLM(DataCollatorForLanguageModeling):
    def torch_call(self, examples: List[Union[List[int], Any, Dict[str, Any]]]) -> Dict[str, Any]:
        batch = super().torch_call(examples)

        # The prompt ends with the response key plus a newline.  We encode this and then try to find it in the
        # sequence of tokens.  This should just be a single token.
        response_token_ids = self.tokenizer.encode(RESPONSE_KEY_NL)

        labels = batch["labels"].clone()

        for i in range(len(examples)):

            response_token_ids_start_idx = None
            for idx in np.where(batch["labels"][i] == response_token_ids[0])[0]:
                response_token_ids_start_idx = idx
                break

            if response_token_ids_start_idx is None:
                raise RuntimeError(
                    f'Could not find response key {response_token_ids} in token IDs {batch["labels"][i]}'
                )

            response_token_ids_end_idx = response_token_ids_start_idx + 1

            # Make pytorch loss function ignore all tokens up through the end of the response key
            labels[i, :response_token_ids_end_idx] = -100

        batch["labels"] = labels

        return batch

In [13]:
if torch.cuda.get_device_name(0) in ["A100-SXM4-40GB", "A100-SXM4-80GB"]:
        torch.backends.cuda.matmul.allow_tf32 = True 
# https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html
data_collator = DataCollatorForCompletionOnlyLM(
    tokenizer = pythia_tokenizer,
    mlm=False, 
    return_tensors="pt", 
    pad_to_multiple_of =   64 if torch.cuda.get_device_name(0) in ["A100-SXM4-40GB", "A100-SXM4-80GB"] else 8
)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_504919/2571383233.py:1 in <module>                                                │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_504919/2571383233.py'                       │
│                                                                                                  │
│ /home/null/miniconda3/envs/temoctalk/lib/python3.10/site-packages/torch/cuda/__init__.py:365 in  │
│ get_device_name                                                                                  │
│                                                                                                  │
│    362 │   Returns:                                                                              │
│    363 │   │   str: the name of the device                                                       │
│    364 │   """                                                                                   │
│ ❱  365 │   return get_device_properties(device).name                                             │
│    366                                                                                           │
│    367                                                                                           │
│    368 def get_device_capability(device: Optional[_device_t] = None) -> Tuple[int, int]:         │
│                                                                                                  │
│ /home/null/miniconda3/envs/temoctalk/lib/python3.10/site-packages/torch/cuda/__init__.py:395 in  │
│ get_device_properties                                                                            │
│                                                                                                  │
│    392 │   Returns:                                                                              │
│    393 │   │   _CudaDeviceProperties: the properties of the device                               │
│    394 │   """                                                                                   │
│ ❱  395 │   _lazy_init()  # will define _get_device_properties                                    │
│    396 │   device = _get_device_index(device, optional=True)                                     │
│    397 │   if device < 0 or device >= device_count():                                            │
│    398 │   │   raise AssertionError("Invalid device id")                                         │
│                                                                                                  │
│ /home/null/miniconda3/envs/temoctalk/lib/python3.10/site-packages/torch/cuda/__init__.py:247 in  │
│ _lazy_init                                                                                       │
│                                                                                                  │
│    244 │   │   # are found or any other error occurs                                             │
│    245 │   │   if 'CUDA_MODULE_LOADING' not in os.environ:                                       │
│    246 │   │   │   os.environ['CUDA_MODULE_LOADING'] = 'LAZY'                                    │
│ ❱  247 │   │   torch._C._cuda_init()                                                             │
│    248 │   │   # Some of the queued calls may reentrantly call _lazy_init();                     │
│    249 │   │   # we need to just return without initializing in that case.                       │
│    250 │   │   # However, we must not let any *other* threads in!                                │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a 
driver from http://www.nvidia.com/Download/index.as

# Download the Model
1. Torch Datat

## Setup Bits and Butes Config

## 4 Bit Configuration

In [14]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "EleutherAI/gpt-neox-20b"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    load_in_8bit = False,
    llm_int8_threshold = 6.0,
    llm_int8_has_fp16_weight=False,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype = torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16,
)


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_504919/2926075251.py:12 in <module>                                               │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_504919/2926075251.py'                       │
│                                                                                                  │
│ /home/null/miniconda3/envs/temoctalk/lib/python3.10/site-packages/torch/cuda/__init__.py:122 in  │
│ is_bf16_supported                                                                                │
│                                                                                                  │
│    119 │   │   cuda_maj_decide = int(cu_vers.split('.')[0]) >= 11                                │
│    120 │   else:                                                                                 │
│    121 │   │   cuda_maj_decide = False                                                           │
│ ❱  122 │   return torch.cuda.get_device_properties(torch.cuda.current_device()).major >= 8 and   │
│    123                                                                                           │
│    124 def _sleep(cycles):                                                                       │
│    125 │   torch._C._cuda_sleep(cycles)                                                          │
│                                                                                                  │
│ /home/null/miniconda3/envs/temoctalk/lib/python3.10/site-packages/torch/cuda/__init__.py:674 in  │
│ current_device                                                                                   │
│                                                                                                  │
│    671                                                                                           │
│    672 def current_device() -> int:                                                              │
│    673 │   r"""Returns the index of a currently selected device."""                              │
│ ❱  674 │   _lazy_init()                                                                          │
│    675 │   return torch._C._cuda_getDevice()                                                     │
│    676                                                                                           │
│    677                                                                                           │
│                                                                                                  │
│ /home/null/miniconda3/envs/temoctalk/lib/python3.10/site-packages/torch/cuda/__init__.py:247 in  │
│ _lazy_init                                                                                       │
│                                                                                                  │
│    244 │   │   # are found or any other error occurs                                             │
│    245 │   │   if 'CUDA_MODULE_LOADING' not in os.environ:                                       │
│    246 │   │   │   os.environ['CUDA_MODULE_LOADING'] = 'LAZY'                                    │
│ ❱  247 │   │   torch._C._cuda_init()                                                             │
│    248 │   │   # Some of the queued calls may reentrantly call _lazy_init();                     │
│    249 │   │   # we need to just return without initializing in that case.                       │
│    250 │   │   # However, we must not let any *other* threads in!                                │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a 
driver from http://www.nvidia.com/Download/index.as

## Download the LM Models
Then we have to apply some preprocessing to the model to prepare it for training. For that use the `prepare_model_for_kbit_training` method from PEFT.

In [15]:
from transformers import AutoModelForCausalLM
assert torch.cuda.is_available(), "You need to have a GPU to run this notebook."
n_gpus = torch.cuda.device_count()

free_in_GB = int(torch.cuda.mem_get_info()[0]/1024**3)
max_memory = f'{int(torch.cuda.mem_get_info()[0]/1024**3)-2}GB'

n_gpus = torch.cuda.device_count()
max_memory = {i: max_memory for i in range(n_gpus)}
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path = PRETRAINED_MODEL_NAME_OR_PATH,
    trust_remote_code = True,
    use_cache = False,
    torch_dtype =  torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16,
    device_map = "auto",
    load_in_8bit = False,
    load_in_4bit = True,
    low_cpu_mem_usage = True,
    max_memory =  max_memory,
    quantization_config = bnb_config,
)


print(model)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_504919/2846471010.py:2 in <module>                                                │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_504919/2846471010.py'                       │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AssertionError: You need to have a GPU to run this notebook.

In [16]:
from peft import prepare_model_for_kbit_training
def print_trainable_parameters(args, model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    if args.bits == 4: trainable_params /= 2
    print(
        f"trainable params: {trainable_params} || "
        f"all params: {all_param} || "
        f"trainable: {100 * trainable_params / all_param}"
    )

model.gradient_checkpointing_enable()
try:
    model = prepare_model_for_kbit_training(model)
except:
    print("Model is already prepared for kbit training")


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/null/miniconda3/envs/temoctalk/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so
/home/null/miniconda3/envs/temoctalk/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32
CUDA SETUP: Loading binary /home/null/miniconda3/envs/temoctalk/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so...


/home/null/miniconda3/envs/temoctalk/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_504919/3035329852.py:19 in <module>                                               │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_504919/3035329852.py'                       │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'model' is not defined

## Set up the LoRa Models

In [17]:
from peft import LoraConfig, get_peft_model
import bitsandbytes as bnb
def find_all_linear_names( model):
    cls = bnb.nn.Linear4bit if BITS == 4 else (bnb.nn.Linear8bitLt if BITS == 8 else torch.nn.Linear)
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])


    if 'lm_head' in lora_module_names: # needed for 16-bit
        lora_module_names.remove('lm_head')
    return list(lora_module_names)

config = LoraConfig(
    r=8, 
    lora_alpha=32, 
    target_modules = find_all_linear_names(  model),
    lora_dropout=0.05, 
    bias="none", 
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
try:
    print_trainable_parameters(model)
except:
    print("Model is already prepared for kbit training")

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_504919/701706016.py:19 in <module>                                                │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_504919/701706016.py'                        │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'model' is not defined

# Training Arguments 
1. WarmUp Steps 
2. Learning Rate 
   1.  0.0001 <path_or_name> https://github.com/artidoro/qlora/tree/main if the model is bigger than 13 Billions

In [24]:
from transformers import Trainer, TrainingArguments , EarlyStoppingCallback
batch_size: int = 8 # if the model is too large for your GPU, decrease this number
micro_batch_size: int = 4
gradient_accumulation_steps = batch_size // micro_batch_size
learning_rate: float = 0.0002
epochs = 32 
print(f"learning_rate: {learning_rate}")
trainer_arguments = TrainingArguments(
            output_dir = "outputs",
            per_device_train_batch_size = batch_size,
            per_device_eval_batch_size = batch_size,
            gradient_accumulation_steps=gradient_accumulation_steps,
            num_train_epochs = epochs,
            learning_rate=learning_rate,
            lr_scheduler_type = "linear",
            warmup_ratio= .3,
            bf16 =  True if torch.cuda.is_bf16_supported() else False,
            fp16 =   False if  torch.cuda.is_bf16_supported() else True,
            logging_steps = 20, # we will log every 20 steps
            eval_steps = 20, # we will evaluate every 20 steps
            evaluation_strategy="steps",
            optim = "paged_adamw_8bit",
            save_strategy="steps",
            report_to = "wandb",
            load_best_model_at_end = True
)

learning_rate: 0.0002

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_504919/1419938607.py:17 in <module>                                               │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_504919/1419938607.py'                       │
│                                                                                                  │
│ /home/null/miniconda3/envs/temoctalk/lib/python3.10/site-packages/torch/cuda/__init__.py:122 in  │
│ is_bf16_supported                                                                                │
│                                                                                                  │
│    119 │   │   cuda_maj_decide = int(cu_vers.split('.')[0]) >= 11                                │
│    120 │   else:                                                                                 │
│    121 │   │   cuda_maj_decide = False                                                           │
│ ❱  122 │   return torch.cuda.get_device_properties(torch.cuda.current_device()).major >= 8 and   │
│    123                                                                                           │
│    124 def _sleep(cycles):                                                                       │
│    125 │   torch._C._cuda_sleep(cycles)                                                          │
│                                                                                                  │
│ /home/null/miniconda3/envs/temoctalk/lib/python3.10/site-packages/torch/cuda/__init__.py:674 in  │
│ current_device                                                                                   │
│                                                                                                  │
│    671                                                                                           │
│    672 def current_device() -> int:                                                              │
│    673 │   r"""Returns the index of a currently selected device."""                              │
│ ❱  674 │   _lazy_init()                                                                          │
│    675 │   return torch._C._cuda_getDevice()                                                     │
│    676                                                                                           │
│    677                                                                                           │
│                                                                                                  │
│ /home/null/miniconda3/envs/temoctalk/lib/python3.10/site-packages/torch/cuda/__init__.py:247 in  │
│ _lazy_init                                                                                       │
│                                                                                                  │
│    244 │   │   # are found or any other error occurs                                             │
│    245 │   │   if 'CUDA_MODULE_LOADING' not in os.environ:                                       │
│    246 │   │   │   os.environ['CUDA_MODULE_LOADING'] = 'LAZY'                                    │
│ ❱  247 │   │   torch._C._cuda_init()                                                             │
│    248 │   │   # Some of the queued calls may reentrantly call _lazy_init();                     │
│    249 │   │   # we need to just return without initializing in that case.                       │
│    250 │   │   # However, we must not let any *other* threads in!                                │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a 
driver from http://www.nvidia.com/Download/index.as

In [23]:
trainer = Trainer(
    model=model,
    tokenizer = pythia_tokenizer,
    args = trainer_arguments,
    train_dataset=split_dataset["train"],
    eval_dataset=split_dataset["test"],
    data_collator=data_collator,
    callbacks = [ EarlyStoppingCallback(
        early_stopping_patience=3 , 
        )
                 ],
)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_504919/1082532498.py:2 in <module>                                                │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_504919/1082532498.py'                       │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'model' is not defined

# Start training

In [20]:
try:
    logger.info("Training")
    trainer.train()
    wandb.finish()
except Exception as e:
    print(e)

Training failed

# Push the Model to the Hugging Face Model Hub

In [21]:
model.push_to_hub("Rami/prompt_generator")

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_504919/144398057.py:1 in <module>                                                 │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_504919/144398057.py'                        │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'model' is not defined

# References
[1] [Dolly Github](https://github.com/databrickslabs/dolly/blob/5021d941d95dddcf1f00d978d7f944709873f419/training/trainer.py#L138)
[2] https://gist.github.com/Birch-san/57878c4a27cf34f57d3e861865a7d0a2
[3] https://github.com/artidoro/qlora/blob/main/qlora.py 
[4] https://github.com/tloen/alpaca-lora/blob/main/finetune.py 